# Objective:
### Perfom RFM analysis to find out the most valuable customers a business should target for their next campaign based on the transaction data and also find customer lifetime value

## 1) Import relevant libraries and read dataset:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns',100)

In [2]:
df = pd.read_excel("CLV_RFM_Practice_Data.xlsx", sheet_name='Transactions')
df.head()

C:\Users\Josh\anaconda3\envs\SignLangauge\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  """Entry point for launching an IPython kernel.


,transaction_id,product_id,customer_id,recency,transaction_date,transaction_month,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,profit,gender,past_3_years_bike_related_purchases,DOB,age,age category,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,Comparison Date,customer title,State
0,1,2,2950,308.0,2017-02-25,February,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-02 00:00:00,17.87,Male,19.0,1955-01-11 00:00:00,66.771587,70.0,Software Engineer I,Financial Services,Mass Customer,N,Yes,10.0,2017-12-30,Lost,VIC
1,2,3,3120,223.0,2017-05-21,May,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-03 00:00:00,1702.55,Female,89.0,1979-02-04 00:00:00,42.689395,50.0,Clinical Specialist,Health,Mass Customer,N,Yes,10.0,NaT,High Moderate,NSW
2,3,37,402,75.0,2017-10-16,October,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-20 00:00:00,1544.61,Male,9.0,1977-06-03 00:00:00,44.363367,0.0,Desktop Support Technician,Retail,Affluent Customer,N,No,22.0,NaT,Low Moderate,NSW
3,4,88,3135,121.0,2017-08-31,August,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,1998-12-16 00:00:00,817.36,Male,83.0,1962-01-14 00:00:00,59.757888,60.0,Staff Scientist,Financial Services,Mass Customer,N,No,16.0,NaT,Lost,NSW
4,5,78,787,90.0,2017-10-01,October,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,2015-08-10 00:00:00,1055.82,Female,29.0,1996-12-15 00:00:00,24.815422,0.0,0,Retail,Affluent Customer,N,Yes,2.0,NaT,Low Moderate,NSW


In [3]:
df2 = df[['customer_id', 'transaction_date', 'transaction_id', 'list_price']]
df2['transaction_date'] = pd.to_datetime(df2['transaction_date'])

C:\Users\Josh\anaconda3\envs\SignLangauge\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## 2) check for null, missing values and descriptive stats of the dataset:

In [4]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customer_id       20000 non-null  int64         
 1   transaction_date  20000 non-null  datetime64[ns]
 2   transaction_id    20000 non-null  int64         
 3   list_price        20000 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 625.1 KB


In [5]:
df2.isna().sum()

customer_id         0
transaction_date    0
transaction_id      0
list_price          0
dtype: int64

In [6]:
df2.describe()

,customer_id,transaction_id,list_price
count,20000.000000,20000.000000,20000.000000
mean,1738.246050,10000.500000,1107.829449
std,1011.951046,5773.647028,582.825242
min,1.000000,1.000000,12.010000
25%,857.750000,5000.750000,575.270000
50%,1736.000000,10000.500000,1163.890000
75%,2613.000000,15000.250000,1635.300000
max,5034.000000,20000.000000,2091.470000


In [7]:
df2.columns

Index(['customer_id', 'transaction_date', 'transaction_id', 'list_price'], dtype='object')

# 3) RFM analysis: find top 100 most valueable customers

RFM stands for recency, frequency and monetary value and can be used to find a business's most valueable customers by looking into each customers purchase patterns

In [8]:
import datetime as dt

NOW = dt.datetime(2017,12,30)
NOW

datetime.datetime(2017, 12, 30, 0, 0)

In [9]:
rfm_table = df2.groupby('customer_id').agg({'transaction_date': lambda x: (NOW - x.max()).days,
                                         'transaction_id': lambda x: len(x),
                                         'list_price': lambda x: x.sum()})

rfm_table['transaction_date'] = rfm_table['transaction_date'].astype(int)

rfm_table.rename(columns={'transaction_date':'recency',
                         'transaction_id':'frequency',
                         'list_price':'monetary_value'}, inplace=True)

rfm_table

,recency,frequency,monetary_value
customer_id,,,
1,7,11,9084.45
2,128,3,4149.07
3,102,8,9888.23
4,195,2,1047.72
5,16,6,5903.20
...,...,...,...
3497,52,3,3744.07
3498,127,6,5177.06
3499,51,7,7673.48


In [10]:
# quantile table using which all the customers will be ranked:

quantiles = rfm_table.quantile(q=[0.25,0.5,0.75])
quantiles

,recency,frequency,monetary_value
0.25,17.0,4.0,4198.3900
0.50,44.0,6.0,6026.8000
0.75,85.0,7.0,8208.8375


In [11]:
# functions to rank the customers based on R,F and M values:


# x: series
# p: column of quantiles to consider
# d: quantiles table itself


# higher the value of recency, lower is the customer rank
# lower the recency value, the better a customer is
def R_class(x,p,d):
    if x < d[p][0.25]:
        return 1
    elif x < d[p][0.5]:
        return 2
    elif x < d[p][0.75]:
        return 3
    else:
        return 4


# higher the frequency and monetary value of a customer, higher is the rank
# higher the frequency and recency value, the better a customer is
def FM_class(x,p,d):
    if x < d[p][0.25]:
        return 4
    elif x < d[p][0.5]:
        return 3
    elif x < d[p][0.75]:
        return 2
    else:
        return 1   

In [12]:
rfm_table['R_rank'] = rfm_table['recency'].apply(R_class, args=('recency',quantiles,))
rfm_table['F_rank'] = rfm_table['frequency'].apply(FM_class, args=('frequency',quantiles,))
rfm_table['M_rank'] = rfm_table['monetary_value'].apply(FM_class, args=('monetary_value',quantiles,))


rfm_table['RFM value'] = rfm_table['R_rank'].astype(str) + rfm_table['F_rank'].astype(str) + rfm_table['M_rank'].astype(str)
rfm_table['RFM value'] = rfm_table['RFM value'].astype(int)

rfm_table

,recency,frequency,monetary_value,R_rank,F_rank,M_rank,RFM value
customer_id,,,,,,,
1,7,11,9084.45,1,1,1,111
2,128,3,4149.07,4,4,4,444
3,102,8,9888.23,4,1,1,411
4,195,2,1047.72,4,4,4,444
5,16,6,5903.20,1,2,3,123
...,...,...,...,...,...,...,...
3497,52,3,3744.07,3,4,4,344
3498,127,6,5177.06,4,2,3,423
3499,51,7,7673.48,3,1,2,312


In [13]:
# the best customers will be ranked 111:
print("There are ",rfm_table[rfm_table['RFM value'] == 111].shape[0]," customers ranked 111")

There are  258  customers ranked 111


In [14]:
# getting the best customers according to the profits the bring in and sorting by monetary value as we need top 100
rfm_table2 = rfm_table[rfm_table['RFM value'] == 111]
rfm_table2.sort_values('monetary_value', ascending=False, inplace= True)
rfm_table2

C:\Users\Josh\anaconda3\envs\SignLangauge\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,recency,frequency,monetary_value,R_rank,F_rank,M_rank,RFM value
customer_id,,,,,,,
2183,8,14,19071.32,1,1,1,111
1140,6,13,16199.24,1,1,1,111
729,1,10,15826.00,1,1,1,111
1317,10,9,15370.81,1,1,1,111
2762,16,10,15071.26,1,1,1,111
...,...,...,...,...,...,...,...
124,2,7,8272.15,1,1,1,111
3337,6,8,8265.62,1,1,1,111
1770,13,7,8228.41,1,1,1,111


In [15]:
# the top 100 customers:

rfm_table2.head(100)

,recency,frequency,monetary_value,R_rank,F_rank,M_rank,RFM value
customer_id,,,,,,,
2183,8,14,19071.32,1,1,1,111
1140,6,13,16199.24,1,1,1,111
729,1,10,15826.00,1,1,1,111
1317,10,9,15370.81,1,1,1,111
2762,16,10,15071.26,1,1,1,111
...,...,...,...,...,...,...,...
2865,0,7,11030.02,1,1,1,111
2120,2,8,11028.33,1,1,1,111
1748,2,10,11012.26,1,1,1,111


# 4) Customer Lifetime Value:

Customer Lifetime Value is a monetary value that represents the amount of revenue or profit a customer will give the company over the period of the relationship

In [16]:
customer = df.groupby('customer_id').agg({'transaction_date':lambda x: (x.max() - x.min()).days,
                              'transaction_id':lambda x: len(x),
                              'list_price': lambda x: x.sum()})

customer.columns = ['cust lifespan', 'purchase frequency', 'total sale']

customer.head()

,cust lifespan,purchase frequency,total sale
customer_id,,,
1,352,11,9084.45
2,112,3,4149.07
3,208,8,9888.23
4,76,2,1047.72
5,286,6,5903.20


In [17]:
# 1) average purchase value:

average_purchase_value = round((np.sum(customer['total sale'])/ np.sum(customer['purchase frequency'])),2)
print(f"average purchase value of customers: ${average_purchase_value}")


# 2) average purchase frequency:
average_purchase_frequency = round((np.sum(customer['purchase frequency'])/ customer.shape[0]),2)
print(f"average purchase frequency of customers: ${average_purchase_frequency}")


# 3) retention rate and churn:
Retention_rate = customer[customer['purchase frequency']>1].shape[0]/customer.shape[0]
print(f"Retention: {Retention_rate}")
churn = round((1 - Retention_rate), 2)
print(f"Churn: {churn}")


# 4) Customer Value:
customer_value = average_purchase_value * average_purchase_frequency
print("Customer Value: ",customer_value)


# 5) Average customer lifespan: in years
average_customer_lifespan = round((np.mean(customer['cust lifespan'])/365),2)
print("Average Customer Lifespan in years: ",average_customer_lifespan)


# 6) Customer Lifetime Value:
customer_lifetime_value = round(customer_value * average_customer_lifespan,2)
print(f"Customer Lifetime Value: ${customer_lifetime_value}")
print()
print(f"The revenue you can expect an average customer to generate during the course of their relationship is ${customer_lifetime_value}")

average purchase value of customers: $1107.83
average purchase frequency of customers: $5.72
Retention: 0.9859759587864911
Churn: 0.01
Customer Value:  6336.7876
Average Customer Lifespan in years:  0.66
Customer Lifetime Value: $4182.28

The revenue you can expect an average customer to generate during the course of their relationship is $4182.28


In [18]:
# finding CLV for each customer individually:

customer

,cust lifespan,purchase frequency,total sale
customer_id,,,
1,352,11,9084.45
2,112,3,4149.07
3,208,8,9888.23
4,76,2,1047.72
5,286,6,5903.20
...,...,...,...
3497,68,3,3744.07
3498,203,6,5177.06
3499,301,7,7673.48


In [19]:
# how much revenue can be expected from each customer in one year:

customer['avg purchase value'] = customer['total sale']/customer['purchase frequency']
customer['customer value'] = customer['avg purchase value'] * customer['purchase frequency']
customer['customer lifetme value'] = customer['customer value'] * (customer['cust lifespan']/365)
customer

,cust lifespan,purchase frequency,total sale,avg purchase value,customer value,customer lifetme value
customer_id,,,,,,
1,352,11,9084.45,825.859091,9084.45,8760.894247
2,112,3,4149.07,1383.023333,4149.07,1273.139288
3,208,8,9888.23,1236.028750,9888.23,5634.936548
4,76,2,1047.72,523.860000,1047.72,218.155397
5,286,6,5903.20,983.866667,5903.20,4625.521096
...,...,...,...,...,...,...
3497,68,3,3744.07,1248.023333,3744.07,697.525370
3498,203,6,5177.06,862.843333,5177.06,2879.296384
3499,301,7,7673.48,1096.211429,7673.48,6327.993096


In [22]:
# most valueable customers as per individual CLV:
customer.sort_values('customer lifetme value', ascending=False).head(100)

,cust lifespan,purchase frequency,total sale,avg purchase value,customer value,customer lifetme value
customer_id,,,,,,
1597,327,12,18052.68,1504.390000,18052.68,16173.222904
941,324,10,17898.46,1789.846000,17898.46,15887.948055
1302,325,13,17035.83,1310.448462,17035.83,15168.889726
2183,289,14,19071.32,1362.237143,19071.32,15100.305425
1140,337,13,16199.24,1246.095385,16199.24,14956.558575
...,...,...,...,...,...,...
1558,294,9,13587.27,1509.696667,13587.27,10944.266795
1625,311,8,12809.37,1601.171250,12809.37,10914.285123
1722,313,9,12696.87,1410.763333,12696.87,10888.000849


In [27]:
# calculating Customer Lifetime Value on a monthly basis:
# this will show how much sum of total revenue one can expect from all customers on a monthly basis



df2['transaction_date2'] = df2['transaction_date']
df2

C:\Users\Josh\anaconda3\envs\SignLangauge\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,customer_id,transaction_date,transaction_id,list_price,transaction_date2
0,2950,2017-02-25,1,71.49,2017-02-25
1,3120,2017-05-21,2,2091.47,2017-05-21
2,402,2017-10-16,3,1793.43,2017-10-16
3,3135,2017-08-31,4,1198.46,2017-08-31
4,787,2017-10-01,5,1765.30,2017-10-01
...,...,...,...,...,...
19995,1018,2017-06-24,19996,2005.66,2017-06-24
19996,127,2017-11-09,19997,416.98,2017-11-09
19997,2284,2017-04-14,19998,1636.90,2017-04-14
19998,2764,2017-07-03,19999,227.88,2017-07-03


In [29]:
customer2 = df2.groupby('customer_id').agg({'transaction_date':lambda x: x.min().month,
                                           'transaction_date2':lambda x: x.min().year,
                              'transaction_id':lambda x: len(x),
                              'list_price': lambda x: x.sum()})

customer2.columns = ['start month','start year', 'purchase frequency', 'total sale']

customer2.head()

,start month,start year,purchase frequency,total sale
customer_id,,,,
1,1,2017,11,9084.45
2,5,2017,3,4149.07
3,2,2017,8,9888.23
4,4,2017,2,1047.72
5,3,2017,6,5903.20


In [51]:
# calculating clv for each month:
# we won't do year because the data is from 2017 only:

months = ['Jan', 'Feb', 'March', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
monthly_avg_sales = []
monthly_purchase_freq =[]
monthly_churn = []
monthly_clv = []

for i in range(1,13):
    customer_m = customer2[customer2['start month'] == i]
    #print("Month: ",months[i-1])
    
    Average_sales_m = round(np.mean(customer_m['total sale']),2)
    #print("Average sales: ",Average_sales_m)
    monthly_avg_sales.append(Average_sales_m)
    
    Purchase_freq_m = round(np.mean(customer_m['purchase frequency']), 2)
    #print("Purchase Frequency: ",Purchase_freq_m)
    monthly_purchase_freq.append(Purchase_freq_m)
    
    Retention_rate_m = customer_m[customer_m['purchase frequency']>1].shape[0]/customer_m.shape[0]
    churn_m = round((1 - Retention_rate), 2)
    #print("churn_m: ", churn_m)
    monthly_churn.append(churn_m)
    
    clv_m = np.round(((Average_sales_m * Purchase_freq_m)/ churn_m) * 0.05, 2)
    #print("CLV: ",clv_m)
    monthly_clv.append(clv_m)

In [52]:
monthly_clv_df = pd.DataFrame(zip(months, monthly_avg_sales, monthly_purchase_freq, monthly_churn, monthly_clv), 
                              columns=['Months', 'Monthly avg sales','Monthly purchase freq','Monthly churn','Monthly CLV'])

display(monthly_clv_df.style.background_gradient())

,Months,Monthly avg sales,Monthly purchase freq,Monthly churn,Monthly CLV
0,Jan,7096.530000,6.410000,0.010000,227443.790000
1,Feb,6685.650000,6.050000,0.010000,202240.910000
2,March,6151.290000,5.540000,0.010000,170390.730000
3,Apr,5761.020000,5.230000,0.010000,150650.670000
4,May,5208.710000,4.590000,0.010000,119539.890000
5,Jun,4775.430000,4.240000,0.010000,101239.120000
6,Jul,3944.460000,3.620000,0.010000,71394.730000
7,Aug,3391.360000,3.060000,0.010000,51887.810000
8,Sep,2763.650000,2.520000,0.010000,34821.990000
9,Oct,2202.710000,2.100000,0.010000,23128.460000
